##Библиотека Surprise

Библиотека Surprise используется для создания рекомендательных систем. Рекомендательная система (англ. recommender system) - это подкласс системы фильтрации информации, которая стремится предсказать некоторый рейтинг или уровень предпочтения, который пользователь соотнесёт с конкретным элементом (например, товаром, фильмом, книгой и т.д.).

**Коллаборативная фильтрация**, совместная фильтрация (англ. collaborative filtering) — метод, использующий известные предпочтения (оценки) группы пользователей для прогнозирования неизвестных предпочтений другого пользователя. Его основное допущение состоит в следующем: те, кто одинаково оценивал какие-либо предметы в прошлом, склонны давать похожие оценки другим предметам и в будущем. Например, с помощью коллаборативной фильтрации музыкальное приложение способно прогнозировать, какая музыка понравится пользователю, имея неполный список его предпочтений (симпатий и антипатий).

В данном туториале рассмотрим построение рекомендательной системы основанной на коллаборативной фильтрации.


In [23]:
! pip install scikit-surprise
from surprise import Dataset
from surprise import Reader

import os
import csv

# Загрузим датасет с рейтингами фильмов
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip
!ls

--2021-11-21 17:57:27--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip.1’

ml-latest-small.zip 100%[===================>] 955.28K  4.54MB/s    in 0.2s    

2021-11-21 17:57:27 (4.54 MB/s) - ‘ml-latest-small.zip.1’ saved [978202/978202]

Archive:  ml-latest-small.zip
replace ml-latest-small/links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  
ml-latest-small  ml-latest-small.zip  ml-latest-small.zip.1  sample_data


Сформируем датасет

In [44]:
def load_dataset():
    reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
    ratings_dataset = Dataset.load_from_file('ml-latest-small/ratings.csv', reader=reader)

    movieID_to_name = {}
    with open('ml-latest-small/movies.csv', newline='', encoding='ISO-8859-1') as csvfile:
            movie_reader = csv.reader(csvfile)
            next(movie_reader)
            for row in movie_reader:
                movieID = int(row[0])
                movie_name = row[1]
                movieID_to_name[movieID] = movie_name
    
    return (ratings_dataset, movieID_to_name)

dataset, movieID_to_name = load_dataset()

Небольшая утилита для получения названия фильма

In [45]:
def getMovieName(movieID):
  if int(movieID) in movieID_to_name:
    return movieID_to_name[int(movieID)]
  else:
      return ""

Разделим выборку на тренировочную и тестовую

In [46]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(dataset, test_size=0.25)

Импортируем и обучим модель

In [47]:
from surprise import SVD
svd = SVD()
svd.fit(trainset)

Воспользуемся тестовой выборкой и рассчитаем среднеквадратическую ошибку

In [49]:
predictions = svd.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8783


0.8783167155506799

Порекомендуем пользователю фильм, предположив, что он оценит его больше чем на 4 балла

In [64]:
for prediction in predictions:
  if prediction.est >= 4:
    print("Можем порекомендовать пользователю {name} фильм {movie}".format(name=prediction.uid, movie=getMovieName(prediction.iid)))
    break

Можем порекомендовать пользователю 98 фильм The Martian (2015)


# Резюме
В данном туториале была построена и обучена рекомендательная система основанная на коллаборативной фильтрации.

# Источники

1.  [Surprise’ documentation](https://surprise.readthedocs.io/en/stable/)
2.  [Wiki Recommender system](https://en.wikipedia.org/wiki/Recommender_system)
3.  [How you can build simple recommender systems with Surprise.](https://towardsdatascience.com/how-you-can-build-simple-recommender-systems-with-surprise-b0d32a8e4802)
4.  [Youtube - How To Use The Surprise Library For Recommendation Engines](https://www.youtube.com/watch?v=fEd1p8-3S7w)
5.  [Build a Recommendation Engine With Collaborative Filtering](https://realpython.com/build-recommendation-engine-collaborative-filtering/#the-dataset)